# Initial Framework RAG Model Support

## Pre-requisites

In [1]:
%pip install -q qdrant-client

Note: you may need to restart the kernel to use updated packages.


In [2]:
# load openai api key
import os

from dotenv import load_dotenv
load_dotenv()

if not 'OPENAI_API_KEY' in os.environ:
    raise ValueError('OPENAI_API_KEY is not set')

## Embedding Model Selection

First let's setup our embedding model and run some tests to make sure its working well.

In [3]:
from openai import OpenAI

from validmind.models import FunctionalModel

client = OpenAI()


def embed(x):
    """Returns a text embedding for the given text"""
    return client.embeddings.create(
        input=x,
        model="text-embedding-3-small",
    ).data[0].embedding

vm_embedder = FunctionalModel(input_id="vm_embedder", predict_fn=embed)

In [4]:
import pandas as pd

import validmind as vm

test_df = pd.DataFrame({
    "text": [
        "I am a software engineer",
        "Hello, world!",
        "I am a data scientist",
        "I am a data scientist. This is a really long example... Just to test what happens.",
    ]
})
vm_test_ds = vm.init_dataset(test_df, text_column="text", __log=False)

2024-04-30 01:45:11,372 - INFO(validmind.client): Pandas dataset detected. Initializing VM Dataset instance...


In [5]:
from validmind.tests import run_test

result = run_test(
    "validmind.model_validation.embeddings.StabilityAnalysisRandomNoise",
    inputs={"model": vm_embedder, "dataset": vm_test_ds},
    params={"probability": 0.8},
)

## Setup Vector Store

#### Load RFP Question/Answer Dataset

In [6]:
# load documents
import os
from csv import DictReader
from uuid import uuid4

## grab all csv files with the name  "rfp_existing_questions_<client_name>.csv"
documents = []
root_dir = "datasets/rag/"
for file in os.listdir(root_dir):
    if file.startswith("rfp_existing_questions_") and file.endswith(".csv"):
        # use csv dict reader to load the csv file
        with open(os.path.join(root_dir, file)) as f:
            reader = DictReader(f)
            for row in reader:
                # add a unique id to the row
                row["id"] = str(uuid4())
                documents.append(row)

documents = documents[:10]

#### Generate embeddings for the questions

In [7]:
from tqdm import tqdm

for doc in tqdm(documents):
    doc["embedding_question"] = embed(doc["RFP_Question"])
    doc["embedding_answer"] = embed(doc["RFP_Answer"])

100%|██████████| 10/10 [00:03<00:00,  2.62it/s]


#### Setup Vector DB

In [8]:
from qdrant_client import QdrantClient

qdrant = QdrantClient(":memory:")

#### Insert embeddings and questions into Vector DB

In [9]:
from qdrant_client.models import Distance, PointStruct, VectorParams

qdrant.recreate_collection(
    "my_rag_collection",
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE),
)

qdrant.upsert(
    "my_rag_collection",
    points=[
        PointStruct(
            id=doc["id"],
            vector=doc["embedding_question"],
            payload={
                k: v
                for k, v in doc.items()
                if k not in ["embedding_question", "embedding_answer"]
            },
        )
        for doc in documents
    ],
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

## Setup Retrieval Model

In [10]:
def retrieve(x_embed, limit=10):
    context = ""

    for result in qdrant.search(
        "my_rag_collection",
        query_vector=x_embed,
        limit=limit,
    ):
        context += f"Q: {result.payload['RFP_Question']}\n"
        context += f"A: {result.payload['RFP_Answer']}\n"
        context += f"Project: {result.payload['Project_Title']}\n\n"

    return context

vm_retriever = FunctionalModel(input_id="vm_retriever", predict_fn=retrieve)

## Setup Generation Model

In [11]:
system_prompt = """
You are an expert RFP AI assistant.
You are tasked with answering new RFP questions based on existing RFP questions and answers.
You will be provided with the existing RFP questions and answer pairs that are the most relevant to the new RFP question.
After that you will be provided with a new RFP question.
You will generate an answer and respond only with the answer.
Ignore your pre-existing knowledge and answer the question based on the provided context.
""".strip()


def generate(x, context):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": context},
            {"role": "user", "content": x},
        ],
    )

    return response.choices[0].message.content

vm_generator = FunctionalModel(input_id="vm_generator", predict_fn=generate)

## Setup RAG Model (Pipeline of "Component" Models)

In [12]:
from validmind.models import RAGModel

vm_rag_model = RAGModel(
    embedder=vm_embedder,
    retriever=vm_retriever,
    generator=vm_generator,
    input_id="vm_rag_model",
)

In [13]:
vm_rag_model.predict([["What is your experience with AI?"]])

[['Our company has 15 years of experience in developing AI-based applications, with a strong portfolio in sectors such as healthcare, finance, and education. For example, one of our notable projects, MediAI Insight for the healthcare industry, showcased significant advancements in patient data analysis, leading to a 30% reduction in diagnostic errors and a 40% improvement in treatment personalization. With over 200 healthcare facilities utilizing our platform and achieving a user satisfaction rate of 95%, we have a proven track record of success in AI application development.']]